In [ ]:
%defaultDatasource jdbc:h2:mem:db

In [ ]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [ ]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
DROP VIEW IF EXISTS interacao;

CREATE VIEW interacao AS
    SELECT da.codeDrug A, db.codeDrug B, count(*) uso_combinado
    FROM DrugUse da, DrugUse db
    WHERE db.codeDrug > da.codeDrug AND db.IdPerson = da.IdPerson
    GROUP BY A, B;

CREATE VIEW interacao30P AS
    SELECT * FROM interacao 
    WHERE uso_combinado > 30;

SELECT * FROM interacao30P;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [ ]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [ ]:
SELECT druga, drugb, weight, (CAST(weight AS DOUBLE)/ (SELECT COUNT(*) FROM DrugInteraction)) suporte
       FROM drugInteraction
       WHERE weight > 30
       ORDER BY suporte;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [ ]:
SELECT *,cast(weight AS DOUBLE)/soma confianca FROM

((SELECT di.druga, di.drugb, di.weight,total.soma
       FROM (SELECT * FROM DrugInteraction) di,
            (SELECT druga,SUM(sa) soma 
                 FROM ((SELECT druga, SUM(weight) sa FROM DrugInteraction GROUP BY druga) UNION (SELECT drugb, SUM(weight) sb FROM DrugInteraction GROUP BY drugb)) 
                 GROUP BY druga) total
       WHERE di.druga = total.druga
       GROUP BY di.druga, di.drugb)
UNION
(SELECT di.druga, di.drugb, di.weight,total.soma
       FROM (SELECT * FROM DrugInteraction) di,
            (SELECT druga,SUM(sa) soma 
                 FROM ((SELECT druga, SUM(weight) sa FROM DrugInteraction GROUP BY druga) UNION (SELECT drugb, SUM(weight) sb FROM DrugInteraction GROUP BY drugb)) 
                 GROUP BY druga) total
       WHERE di.drugb = total.druga
       GROUP BY di.druga, di.drugb
))       
WHERE weight > 30 ORDER BY confianca;


# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [ ]:
SELECT druga, drugb, weight, (CAST(weight AS DOUBLE)/ (SELECT COUNT(*) FROM DrugInteraction)) confianca
       FROM drugInteraction
       WHERE weight > 30
       ORDER BY confianca;